In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
import torch.optim as optim


data_path = './csv/koniq10k_merged.csv'
data = pd.read_csv(data_path)
data = data.dropna()  
features = data[['arniqa', 'clipiqa', 'cnniqa', 'metaiqa']]
target = data['overall_score']
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(features)
X_train, X_val, y_train, y_val = train_test_split(scaled_features, target, test_size=0.2, random_state=42)
X_train = torch.tensor(X_train, dtype=torch.float32)
X_val = torch.tensor(X_val, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.float32).unsqueeze(1)
y_val = torch.tensor(y_val.values, dtype=torch.float32).unsqueeze(1)


print(data.describe())
print(features.describe())
print(target.describe())
import torch
import torch.nn as nn

class MLPModel(nn.Module):
    def __init__(self):
        super(MLPModel, self).__init__()
        self.fc1 = nn.Linear(4, 64) 
        self.fc2 = nn.Linear(64, 32) 
        self.fc3 = nn.Linear(32, 1)  
        
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        x = self.sigmoid(x)
        
        return x


model = MLPModel()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


epochs = 20 # エポック数
batch_size = 128  # バッチサイズ

train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
val_dataset = torch.utils.data.TensorDataset(X_val, y_val)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    train_loss /= len(train_loader)

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            val_loss += loss.item()

    val_loss /= len(val_loader)
    print(f"Epoch {epoch + 1}/{epochs}, Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}")

# モデルの評価
def evaluate_model(model, loader):
    model.eval()
    total_loss = 0.0
    with torch.no_grad():
        for X_batch, y_batch in loader:
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            total_loss += loss.item()
    return total_loss / len(loader)

final_val_loss = evaluate_model(model, val_loader)
print(f"Final Validation Loss (MSE): {final_val_loss:.4f}")


             arniqa       clipiqa        cnniqa       metaiqa  overall_score
count  10073.000000  10073.000000  10073.000000  10073.000000   10073.000000
mean       0.583761      0.485806      0.546892      0.637708       0.587296
std        0.129984      0.159973      0.140182      0.082479       0.154324
min        0.046694      0.060973      0.118904      0.344933       0.039118
25%        0.503696      0.360822      0.431291      0.583008       0.492478
50%        0.618319      0.485717      0.594100      0.634901       0.623544
75%        0.683933      0.610258      0.667582      0.689869       0.707125
max        0.832535      0.890501      0.741739      0.939210       0.883889
             arniqa       clipiqa        cnniqa       metaiqa
count  10073.000000  10073.000000  10073.000000  10073.000000
mean       0.583761      0.485806      0.546892      0.637708
std        0.129984      0.159973      0.140182      0.082479
min        0.046694      0.060973      0.118904      0.3449